In [1]:
!python3 -m pip install --upgrade pip

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
# !pip install openai==0.27.8
!pip install openai==0.28.1

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip install python-dotenv tiktoken

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
!pip install PyPDF2

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
!pip install pdf2image

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 3 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.1
Err:1 http://security.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.1
  404  Not Found [IP: 91.189.91.81 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/p/poppler/poppler-utils_22.02.0-2ubuntu0.1_amd64.deb  404  Not Found [IP: 91.189.91.81 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [7]:
!pip install PyMuPDF==1.23.5

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [8]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [33]:
import re
# 取得したいURL
url = "https://arxiv.org/abs/2310.14587"
identifier = re.search(r'/([^/]+)$', url).group(1)
identifier

'2310.14587'

In [34]:
import requests

def download_pdf(link, save_path):
    response = requests.get(link)
    with open(save_path, 'wb') as file:
        file.write(response.content)

In [35]:
import os

files_to_remove = ['sample.pdf']

for file in files_to_remove:
    if os.path.exists(file):
        os.remove(file)
        print(f"{file} を削除しました。")
    else:
        print(f"{file} は存在しません。")

sample.pdf を削除しました。


In [36]:
url = "https://arxiv.org/pdf/" + identifier

# download_pdf(url, nougat_path)
download_pdf(url, "sample.pdf")

In [14]:
pdf_path = "./sample.pdf"

In [37]:
import fitz

# PyMuPDFを使用してPDFファイルから全てのテキストを抽出
def extract_text_with_pymupdf(pdf_path):
    with fitz.open(pdf_path) as doc:
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
    return text

# テキストを抽出
text_from_pdf = extract_text_with_pymupdf(pdf_path)
def split_text_into_sections(text):
    sections = []
    current_section = None
    last_section_number = 0  # 最後に抽出したセクションの番号
    
    # セクションのタイトルと開始位置を探すための正規表現パターン
    section_pattern = re.compile(r'\n(\d+)\n([^\d\n].+?)\n', re.MULTILINE)
    
    # Abstractを探すための正規表現パターン
    abstract_pattern = re.search(r'\nabstract\n(.*?)(?=\n\d+|$)', text.lower(), re.DOTALL)
    if abstract_pattern:
        sections.append({
            'title': 'Abstract',
            'content': abstract_pattern.group(1).strip(),
            'start': abstract_pattern.start(),
            'end': abstract_pattern.end()
        })
    
    for match in section_pattern.finditer(text):
        section_number = int(match.group(1))
        section_title = match.group(2).strip()
        
        # セクション番号が前のセクション番号よりも大きいか確認
        if section_number > last_section_number:
            # "Work in progress"を含まないセクションを抽出
            if "work in progress" not in section_title.lower():
                if current_section:
                    current_section['content'] = text[current_section['end']:match.start()].strip()
                    sections.append(current_section)
                section_start = match.start()
                section_end = match.end()
                current_section = {'title': f"{section_number} {section_title}", 'content': '', 'start': section_start, 'end': section_end}
                last_section_number = section_number
        
    if current_section:
        current_section['content'] = text[current_section['end']:].strip()
        sections.append(current_section)
    
    return sections

# テキストをセクションごとに分割
sections = split_text_into_sections(text_from_pdf)

# 分割されたセクションのタイトルと最初の100文字を表示
[(section['title'], section['content'][:100]) for section in sections]


[('Abstract',
  'modern search engines are built on a stack of different components, including query un-\nderstanding,'),
 ('1 Introduction',
  'With the ever-increasing amount of information on the web, search engines have emerged as an\nindispe'),
 ('3 Large Search Model',
  'First-stage Retrieval\nPre-ranking\nRanking\n~1000 documents \n~100 documents\nBillions of documents\nLarg'),
 ('4 Proof-of-Concept Experiments',
  'In this section, we present some proof-of-concept experiments to showcase the potential of our\npropo'),
 ('9 Table 4. An example for answer generation. Our model generates an answer for the given query “rsa',
  'definition key” conditioned on the top-100 retrieved contexts.\nTable 3 indicates that our model outp')]

In [16]:
import openai
import tiktoken

In [17]:
from dotenv import load_dotenv
import os

In [18]:
load_dotenv()

True

In [19]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [38]:
class ChainOfDensity:
    def __init__(self, model_name="gpt-3.5-turbo"):
        self.model_name = model_name
    
    def num_tokens_from_string(self, string: str) -> int:
        """Returns the number of tokens in a text string."""
        encoding = tiktoken.encoding_for_model(self.model_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens
    
    def create_prompt(self, text):
        """Creates a prompt for the Chain of Density task."""
        return f"""
        Article: {text}

        You will generate increasingly concise, entity-dense summaries of the above Article.

        Repeat the following 2 steps 5 times.

        Step 1. Identify 1-3 informative Entities (";" delimited) from the Article which are missing from the previously generated summary.
        Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the Missing Entities.

        A Missing Entity is:
        - Relevant: to the main story.
        - Specific: descriptive yet concise (5 words or fewer).
        - Novel: not in the previous summary.
        - Faithful: present in the Article.
        - Anywhere: located anywhere in the Article.

        Guidelines:
        - The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
        - Make every word count: re-write the previous summary to improve flow and make space for additional entities.
        - Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
        - The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Article.
        - Missing entities can appear anywhere in the new summary.
        - Never drop entities from the previous summary. If space cannot be made, add fewer new entities. Remember, use the exact same number of words for each summary. 

        Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary"
        Finally, output must be in Japanese.
        """
    
    def generate_summary(self, text):
        """Generates a summary using the Chain of Density method."""
        token_count = self.num_tokens_from_string(text)
        if token_count > 4096:
            print("Warning: Input is too long. Splitting the input into smaller chunks.")
            # 文単位で分割
            sentences = re.split(r'(?<=[.!?]) +', text)
            # 分割した文を4096トークンに収めるようにグループ化
            chunks = []
            current_chunk = ""
            for sentence in sentences:
                if len(current_chunk) + len(sentence) + 1 > 4096:
                    chunks.append(current_chunk)
                    current_chunk = sentence
                else:
                    current_chunk += (" " + sentence)
            chunks.append(current_chunk)
        else:
            chunks = [text]

        # 各チャンクに対してAPIを呼び出し
        responses = []
        for chunk in chunks:
            prompt = self.create_prompt(chunk)
            response = openai.ChatCompletion.create(
                model=self.model_name,
                messages=[
                    {"role" : "system", "content" : "Outputs should be generated in step by step."},
                    {"role": "system", "content": "Given the remaining options, what is the final answer in Japanese?"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.8,
                n=1,
                stop=None,
                top_p=1,
            )
            responses.append(response['choices'][0]['message']['content'])

        # 結果を結合して返す
        formatted_summary = " ".join(responses)
        return formatted_summary
    
    def format_cod_output(self, cod_output):
        """Formats the output of Chain of Density to be a readable text."""
        try:
            summaries = json.loads(cod_output)
            formatted_summaries = " ".join([summary['Denser_Summary'] for summary in summaries])
            return formatted_summaries
        except json.JSONDecodeError:
            print("Error: The output is not in valid JSON format.")
            return None

## アルゴリズム: IEP (Interact-Eliminate-Propose)
1. Planning: 問題を理解し、可能な答えをk個提案します。
2. Inferring: 各オプションについて推論します。
    - オプションが真である場合の前提条件は何かを考えます。
    - 必要に応じて関連情報を自問自答または検索して前提条件を取得します。
    - オプションを仮説として考え、それがこれらの前提条件と一致するかどうかを確認します。
3. Eliminating: 矛盾しているオプションを排除します。
    - もし一致するならば続けます。
    - もし矛盾するならば候補から削除します。
4. Answering: 最終的な答えを提案します。

## 実装ステップ
1. テキストの前処理: 各章のテキストをトークンの最大数に収まるように分割します。
2. 要約の生成: 各章のテキストを使用して要約を生成します。
3. 質問応答: 生成した要約を基に質問応答を行い、答えを絞り込みます。
4. 最終答えの提案: 答えの候補から最も適したものを選び出します。

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


class IEP:
    def __init__(self):
        self.candidates = []
    
    def set_initial_candidates(self, initial_candidates):
        self.candidates = initial_candidates
    
    def planning(self, text):
        # ここでは単純にテキストを要約して候補を生成しています。
        # 実際には、より複雑なロジックや外部の情報を利用して候補を生成することが考えられます。
        prompt = [
            {"role" : "system", "content" : "Outputs should be generated in step by step."},
            # {"role": "system", "content": "Please understand the problem and propose 5 possible answers in short sentences in Japanese."},
            # {"role": "system", "content": "Please understand the problem and propose 5 possible answers in short sentences."},
            {"role": "system", "content": "Please understand the problem and propose multiple possible answers"},
            {"role": "user", "content": text}
        ]
        summary = self._call_openai_api(prompt)
        return summary
    
    def inferring_and_eliminating(self):
        remaining_candidates = self.candidates[:]
        for candidate in self.candidates:
            premise = f"The premise if {candidate} is true is that {candidate} would logically follow from the given information."
            prompt = [
                {"role" : "system", "content" : "Outputs should be generated in step by step."},
                # {"role": "system", "content": "Think the option as hypothesis. Whether it entails with those premises? Please answer in Japanese."},
                {"role": "system", "content": "Think the option as hypothesis. Whether it entails with those premises?"},
                {"role": "user", "content": f"Hypothesis: {candidate}\nPremises: {premise}"}
            ]
            entailment = self._call_openai_api(prompt)
            
            if "矛盾" in entailment or "contradict" in entailment.lower():
                remaining_candidates.remove(candidate)
            # similarity_to_contradiction = calculate_similarity(entailment, "contradict")
            # similarity_to_inconsistency = calculate_similarity(entailment, "矛盾")
            # if similarity_to_contradiction > 0.5 or similarity_to_inconsistency > 0.5:
            #     remaining_candidates.remove(candidate)
                
        self.candidates = remaining_candidates
        return self.candidates
    
    def answering(self):
        if not self.candidates:
            return "答えることができる候補がありません。"
        elif len(self.candidates) == 1:
            return self.candidates[0]
        else:
            prompt = [
                {"role" : "system", "content" : "Outputs should be generated in step by step."},
                {"role": "system", "content": "Given the remaining options, what is the final answer in Japanese?"},
                {"role": "user", "content": ", ".join(self.candidates)}
            ]
            final_answer = self._call_openai_api(prompt)
            return final_answer
    
    def num_tokens_from_string(self, string: str, encoding_name: str) -> int:
        """Returns the number of tokens in a text string."""
        encoding = tiktoken.encoding_for_model(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens
    
    def _call_openai_api(self, prompt):
        model_name = "gpt-3.5-turbo"
        # model="gpt-3.5-turbo-16k",
        # model="gpt-4-0613",
        # 入力メッセージの'user'ロールのテキストを取得
        text = next((message['content'] for message in prompt if message['role'] == 'user'), None)
        if text is None:
            raise ValueError("User's message not found in the prompt.")

        # トークン数をチェック
        token_count = self.num_tokens_from_string(text, model_name)
        chunk_num = 0
        if token_count > 4096:
            print("Warning: Input is too long. Splitting the input into smaller chunks.")
            # 文単位で分割
            sentences = re.split(r'(?<=[.!?]) +', text)
            # 分割した文を4096トークンに収めるようにグループ化
            chunks = []
            current_chunk = ""
            for sentence in sentences:
                if len(current_chunk) + len(sentence) + 1 > 4096:
                    chunks.append(current_chunk)
                    current_chunk = sentence
                else:
                    current_chunk += (" " + sentence)
            chunks.append(current_chunk)
            chunk_num += 1
        else:
            chunks = [text]
            chunk_num += 1
        
        # 各チャンクに対してAPIを呼び出し
        responses = []
        # print(f"チャンク数: {chunk_num}")
        if chunk_num >= 3:
            # チャンク数があまりに多い場合は要約を実行しない
            # CoDで要約してもいいかも
            return text
        for chunk in chunks:
            # プロンプトの'user'ロールのテキストを更新
            for message in prompt:
                if message['role'] == 'user':
                    message['content'] = chunk
            
            response = openai.ChatCompletion.create(
                model=model_name,
                messages=prompt,
                temperature=0.8,
                # max_tokens=800,  # 生成するトークンの最大数（入力と合算して4097以内に収める必要あり）
                n=1,  # 生成するレスポンスの数
                stop=None,  # 停止トークンの設定
                top_p=1,  # トークン選択時の確率閾値
            )
            responses.append(response['choices'][0]['message']['content'])
        
        # 結果を結合して返す
        return " ".join(responses)

In [41]:
# 論文タイトル
with fitz.open(pdf_path) as doc:
    page_num = 0
    page = doc.load_page(page_num)
    text = page.get_text()

# Extract text until "Abstract" or "ABSTRACT"
abstract_index = text.lower().find("abstract")
pdf_title = text[:abstract_index ] if abstract_index != -1 else "Abstract not found"
print(pdf_title)

OPINION PAPER
Large Search Model: Redefining Search Stack
in the Era of LLMs
Liang Wang∗, Nan Yang∗, Xiaolong Huang,
Linjun Yang, Rangan Majumder, Furu Wei
Microsoft Corporation



In [42]:
sections[0]

{'title': 'Abstract',
 'content': 'modern search engines are built on a stack of different components, including query un-\nderstanding, retrieval, multi-stage ranking, and question answering, among others. these\ncomponents are often optimized and deployed independently. in this paper, we introduce\na novel conceptual framework called large search model, which redefines the conventional\nsearch stack by unifying search tasks with one large language model (llm). all tasks are\nformulated as autoregressive text generation problems, allowing for the customization of\ntasks through the use of natural language prompts. this proposed framework capitalizes on\nthe strong language understanding and reasoning capabilities of llms, offering the potential\nto enhance search result quality while simultaneously simplifying the existing cumbersome\nsearch stack. to substantiate the feasibility of this framework, we present a series of proof-\nof-concept experiments and discuss the potential challen

In [24]:
# CoDのインスタンスを作成
cod = ChainOfDensity()

if pdf_title != "Abstract not found":
    # CoDで要約を生成
    formatted_summary = cod.generate_summary(sections[0]['content'])
else:
    formatted_summary = "Abstract not found"

In [25]:
# "Abstract" or "ABSTRACT"の要約
print(formatted_summary)

[
    {
        "Missing_Entities": "search stack; autoregressive text generation",
        "Denser_Summary": "現代の検索エンジンは、検索スタックとして知られる複数のコンポーネントで構築されています。クエリの理解、検索、マルチステージのランキング、質問への回答などが含まれます。この論文では、大規模な検索モデルという新しい概念フレームワークを紹介します。このフレームワークでは、すべてのタスクを自己回帰的なテキスト生成問題として定式化し、自然言語のプロンプトを使用してタスクをカスタマイズすることができます。この提案されたフレームワークは、大規模な言語モデルの強力な言語理解と推論能力を活用し、既存の煩雑な検索スタックを簡素化しながら検索結果の品質を向上させる可能性を提供します。このフレームワークの実現可能性を確認するために、一連の概念実証実験を行い、実世界の検索システムへの実装に関連する潜在的な課題について議論します。"
    },
    {
        "Missing_Entities": "large language model; natural language prompts",
        "Denser_Summary": "大規模な言語モデル（LLM）を使用した新しい概念フレームワークである大規模な検索モデルを紹介します。このフレームワークでは、すべてのタスクを自己回帰的なテキスト生成問題として定式化し、自然言語のプロンプトを使用してタスクをカスタマイズすることができます。この提案されたフレームワークは、大規模な言語モデルの強力な言語理解と推論能力を活用し、既存の煩雑な検索スタックを簡素化しながら検索結果の品質を向上させる可能性を提供します。"
    },
    {
        "Missing_Entities": "proof-of-concept experiments; real-world search systems",
        "Denser_Summary": "提案された大規模な検索モデルの実現可能性を確認するために、一連の概念実証実験を行い、実世界の検索システムへの実装に関連する潜在的な課題について

In [40]:
print(f"sections数: {len(sections)}")

sections数: 5


In [43]:
# 各章に対してIEPを適用
for section in sections[:-1]:
    with Timer(prefix=f'"Chapter: {section["title"]}'):
        # IEPのインスタンスを作成
        iep = IEP()

        # 章ごとの前提とする要約文章の生成
        with Timer(prefix=f'"Candidates: '):
            initial_candidates = iep.planning(section['content']).split("\n")
            # print(f"{initial_candidates}")
            iep.set_initial_candidates(initial_candidates)
        # 計画と生成文章を比較して矛盾がないかを確認
        with Timer(prefix=f'"Remaining Candidates: '):
            remaining_candidates = iep.inferring_and_eliminating()
            # print(f"{remaining_candidates}")
        # 最終回答
        with Timer(prefix=f'"Final Answer: '):
            final_answer = iep.answering()
            print(f"{final_answer}")
    print("\n" + "="*50 + "\n")

"Candidates:  5.266[s]
"Remaining Candidates:  29.084[s]
最終的な答えは、論文はLSMフレームワークを紹介し、大規模な言語モデル（LLM）を使用してすべての検索タスクを統一することを目指していると述べています。このアプローチにより、検索結果の品質が向上し、既存の検索スタックが簡素化される可能性があります。LSMフレームワークの実現可能性は、概念実証実験によって支持されており、実世界の検索システムにおける実装に関連する潜在的な課題も議論されています。
"Final Answer:  3.769[s]
"Chapter: Abstract 38.119[s]


"Candidates:  10.025[s]
"Remaining Candidates:  22.889[s]
最終的な答えは、2番のオプションです。
"Final Answer:  0.586[s]
"Chapter: 1 Introduction 33.501[s]


"Candidates:  6.244[s]
"Remaining Candidates:  22.904[s]
このセクションでは、著者たちは大規模な検索モデルを使用した情報検索タスクの統合的なモデリングアプローチについて議論しています。彼らはこのアプローチを従来の検索スタックと比較し、大規模な検索モデルの課題一般化とカスタマイズ化の利点を強調しています。また、大規模な言語モデル（LLM）の大規模展開に伴うレイテンシとコストの課題についても議論しています。

著者たちは、自然言語プロンプティングを使用して異なる検索タスクに合わせて大規模な検索モデルをカスタマイズすることを提案しています。彼らはLLMがテキストを生成するための2つの方法、コンテキスト内学習とインストラクションについて説明しています。また、異なる情報検索タスクの例と、それらがプロンプトをカスタマイズすることで大規模な検索モデル内でどのように具体化されるかについても説明しています。

著者たちはまた、大規模な検索モデルでの長いコンテキストモデリングの重要性についても強調しています。彼らは現在のLLMが制限された量のコンテキストを処理する能力の制約と、効果的なコンテキストの長さを拡張するための課題について議論しています。

画像の抽出

In [44]:
!pip install fitz

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [45]:
!pip uninstall -y PyMuPDF
!pip install PyMuPDF

Found existing installation: PyMuPDF 1.23.5
Uninstalling PyMuPDF-1.23.5:
  Successfully uninstalled PyMuPDF-1.23.5
  Using cached PyMuPDF-1.23.5-cp310-none-manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached PyMuPDF-1.23.5-cp310-none-manylinux2014_x86_64.whl (4.3 MB)
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [46]:
!pip install fitz

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [47]:
import fitz  # PyMuPDF
from PIL import Image

# pdf_path = nougat_path
pdf_path = "sample.pdf"
doc = fitz.open(pdf_path)
extracted_images = []

for page in doc:
    # ページからイメージを抽出
    for img_index in page.get_images(full=True):
        xref = img_index[0]
        base_image = fitz.Pixmap(doc, xref)
        pil_img = Image.frombytes("RGB", [base_image.width, base_image.height], base_image.samples)
        extracted_images.append(pil_img)

# 最初の抽出された図を表示
if extracted_images:
    extracted_images[0].show()
else:
    "このPDFには抽出可能な図が含まれていません。"


In [48]:
for img in extracted_images:
    img.show()